In [1]:
%env VECLIB_MAXIMUM_THREADS=1
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1

!make clean

import os, platform
import numpy as np
#np.show_config()

env: VECLIB_MAXIMUM_THREADS=1
env: MKL_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1
make -C 01_mul clean
rm -rf *.o *.dSYM/ mul
make -C 02_fma clean
rm -rf *.o *.dSYM/ fma
make -C 03_omp clean
rm -rf *.o *.dSYM/ omp
rm -rf *.o *.dSYM/ 


# SIMD (vector processing)

1. Types of parallelism
   1. Shared-memory parallelism
   2. Distributed-memory parallelism
   3. Vector processing
2. SIMD instructions
   1. CPU capabilities
   2. x86 intrinsic functions
   3. Symbol table
   4. Inspect assembly: 1, 3, 5 multiplications

# Types of parallelism

The popular computer architecture is based on sequential processing.  The most fundamental processing unit executes instructions one by one.

<center><img src="architecture.png" alt="Common computer architecture" /></center>

If we assume the processor can only perform sequential processing, we need to use multiple processors to perform parallel processing.  Differentiated by the memory access, the parallelism can be broadly set to two categories:

* Shared-memory parallel processing
* Distributed-memory parallel processing

## Shared-memory parallel processing

<br />
<center><img src="shared.png" alt="Shared-memory parallelism" /></center>

## Distributed-memory parallel processing

<br />
<center><img src="distributed.png" alt="Distributed-memory parallelism" /></center>

# Vector processing

When the parallelism happens in the processor (one processing unit or core), it is usually done once for a single instruction with multiple data (SIMD).  It has also been called vector processing (it's an illustrative name).

Before showing what is vector processing, let us see the ordinary scalar execution:

<center><img src="scalar.png" alt="Scalar execution" /></center>

The vector execution uses a wider register so that it can perform an operation for multiple data at once:

<center><img src="vector.png" alt="Vector execution" /></center>

# Check CPU capabilities

To take advantage of SIMD, we will need to inspect the CPU instructions, or the assembly.  But most of the time we stay in high-level languages.  The way we deal with the assembly is to get familiar with the instructions, e.g., using [x86 and amd64 instruction reference](https://www.felixcloutier.com/x86/).

x86 provides a series of SIMD instructions, including

* 64-bit: MMX
* 128-bit: SSE, SSE2, SSE3, SSE4, SSE4.1, SSE4.2 (streaming simd extension)
* 256-bit: AVX, AVX2 (advanced vector extension)
* 512-bit: AVX-512

Recent processors usually are equipped with AVX2, which was released with Haswell in 2013.  Before asking the compiler to use the specific instruction set, query the operating system for the cpu capabilities.

In [2]:
print("Check on", platform.system())
if 'Linux' == platform.system():
    # check whether your cpu supports avx2 on linux
    !grep flags /proc/cpuinfo
elif 'Darwin' == platform.system():
    # check whether your cpu supports avx2 on mac
    !sysctl -a | grep machdep.cpu.*features

Check on Darwin
machdep.cpu.features: FPU VME DE PSE TSC MSR PAE MCE CX8 APIC SEP MTRR PGE MCA CMOV PAT PSE36 CLFSH DS ACPI MMX FXSR SSE SSE2 SS HTT TM PBE SSE3 PCLMULQDQ DTES64 MON DSCPL VMX EST TM2 SSSE3 FMA CX16 TPR PDCM SSE4.1 SSE4.2 x2APIC MOVBE POPCNT AES PCID XSAVE OSXSAVE SEGLIM64 TSCTMR AVX1.0 RDRAND F16C
machdep.cpu.leaf7_features: RDWRFSGS TSC_THREAD_OFFSET SGX BMI1 AVX2 SMEP BMI2 ERMS INVPCID FPU_CSDS MPX RDSEED ADX SMAP CLFSOPT IPT MDCLEAR TSXFA IBRS STIBP L1DF SSBD
machdep.cpu.extfeatures: SYSCALL XD 1GBPAGE EM64T LAHF LZCNT PREFETCHW RDTSCP TSCI


# x86 intrinsic functions

Major compilers provide header files for using the intrinsic functions that can be directly translated into the SIMD instructions:

* `<mmintrin.h>`: MMX
* `<xmmintrin.h>`: SSE
* `<emmintrin.h>`: SSE2
* `<pmmintrin.h>`: SSE3
* `<tmmintrin.h>`: SSSE3
* `<smmintrin.h>`: SSE4.1
* `<nmmintrin.h>`: SSE4.2
* `<ammintrin.h>`: SSE4A
* `<immintrin.h>`: AVX
* `<zmmintrin.h>`: AVX512

You may also use `<x86intrin.h>` which includes everything.

With the intrinsic functions, programmers don't need to really write assembly, and can stay in the high-level languages most of the time.  The intrinsic functions correspond to x86 instructions.  An example of using it:

```cpp
__m256 * ma = (__m256 *) (&a[i*width]);
__m256 * mb = (__m256 *) (&b[i*width]);
__m256 * mr = (__m256 *) (&r[i*width]);
*mr = _mm256_mul_ps(*ma, *mb);
```

**Intel intrinsic guide**: Intel maintains a website to show the available intrinsics: https://software.intel.com/sites/landingpage/IntrinsicsGuide/ .  Consult and remember it when needed.

Using intrinsics and SIMD for optimization is a tedious process.  The materials presented here are not a complete guide to you, but show you one way to study and measure the benefits.  The measurement is important to assess whether or not you need the optimization.

We will use the example, `01_mul/mul.cpp`, to show how to use the 256-bit-wide AVX to perform vector multiplication for 8 single-precision floating-point values.

```cpp
constexpr const size_t width = 8;
constexpr const size_t repeat = 1024 * 1024;
constexpr const size_t nelem = width * repeat;
```

In [3]:
# time the difference between the loop and the simd/avx versions
!g++ -std=c++17 -g -O3 -m64 -mavx 01_mul/mul.cpp -o 01_mul/mul ; 01_mul/mul

width: 8
nelem: 8388608

arr: 0x0x101282000
brr: 0x0x103282000
rrr1: 0x0x105282000
rrr2: 0x0x107282000

Timing repeats for 100 times and takes the minimum

1 multiplication by loop takes: 0.00529216 sec
1 multiplication by simd takes: 0.00354195 sec

3 multiplication by loop takes: 0.0110599 sec
3 multiplication by simd takes: 0.00341286 sec

5 multiplication by loop takes: 0.0217136 sec
5 multiplication by simd takes: 0.00384278 sec



## Symbol table

I use [radare2](https://rada.re/n/) to inspect the assembly of the generated image.  Before really checking the assembly, we need to identify what functions to be inspected from the symbol table.

In [4]:
# take a look at the symbol table
!r2 -Aqc "e scr.color=0 ; afl" 01_mul/mul

0x100002620   27 3402         entry0
0x100001690    3 175          sym.multiply1_loop_float__float__float
0x100001740    3 102          sym.multiply1_simd_float__float__float
0x1000017b0    3 352          sym.multiply3_loop_float__float__float
0x100001910    3 107          sym.multiply3_simd_float__float__float
0x100001980    3 540          sym.multiply5_loop_float__float__float
0x100001ba0    3 87           sym.multiply5_simd_float__float__float
0x100001c00   80 1834 -> 1784 sym.run_std::__1::function_void_float__float__float____float__float__float
0x100002380    5 645          sym.check_float__float
0x100003530    6 249          sym.std::__1::basic_ostream_char_std::__1::char_traits_char___std::__1::__put_character_sequence_char_std::__1::char_traits_char___std::__1::basic_ostream_char_std::__1::char_traits_char____charconst__unsignedlong
0x100003906    1 6            sym.std::__1::basic_ostream_char_std::__1::char_traits_char__::operator___unsignedlong
0x100003900    1 6            

## 1 multiplication

To demonstrate the effect of different ratio of calculations to memory access, I use 3 sets of multiplication.  The first set uses 1 multiplication:

```cpp
void multiply1_loop(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat*width; i+=width)
    {
        for (size_t j=i; j<i+width; ++j)
        {
            r[j] = a[j] * b[j];
        }
    }
}

void multiply1_simd(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat; ++i)
    {
        __m256 * ma = (__m256 *) (&a[i*width]);
        __m256 * mb = (__m256 *) (&b[i*width]);
        __m256 * mr = (__m256 *) (&r[i*width]);
        *mr = _mm256_mul_ps(*ma, *mb);
    }
}
```

In [5]:
# 1 multiplication with loop
!r2 -Aqc "e scr.color=0 ; s sym.multiply1_loop_float__float__float ; pdf" 01_mul/mul

            ;-- func.100001690:
/ (fcn) sym.multiply1_loop_float__float__float 175
|   sym.multiply1_loop_float__float__float ();
|           ; DATA XREF from entry0 (0x100002bf0)
|           0x100001690      55             push rbp
|           0x100001691      4889e5         mov rbp, rsp
|           0x100001694      31c0           xor eax, eax
|           0x100001696      662e0f1f8400.  nop word cs:[rax + rax]
|           ; CODE XREF from sym.multiply1_loop_float__float__float (0x100001737)
|       .-> 0x1000016a0      c5fa100487     vmovss xmm0, dword [rdi + rax*4]
|       :   0x1000016a5      c5fa590486     vmulss xmm0, xmm0, dword [rsi + rax*4]
|       :   0x1000016aa      c5fa110482     vmovss dword [rdx + rax*4], xmm0
|       :   0x1000016af      c5fa10448704   vmovss xmm0, dword [rdi + rax*4 + 4]
|       :   0x1000016b5      c5fa59448604   vmulss xmm0, xmm0, dword [rsi + rax*4 + 4]
|       :   0x1000016bb      c5fa11448204   vmovss dword [rdx + rax*4 + 4], xmm0
|       :   0x100

In [6]:
# 1 multiplication with simd/avx
!r2 -Aqc "e scr.color=0 ; s sym.multiply1_simd_float__float__float ; pdf" 01_mul/mul

            ;-- func.100001740:
/ (fcn) sym.multiply1_simd_float__float__float 102
|   sym.multiply1_simd_float__float__float ();
|           ; DATA XREF from entry0 (0x100002ce3)
|           0x100001740      55             push rbp
|           0x100001741      4889e5         mov rbp, rsp
|           0x100001744      31c0           xor eax, eax
|           0x100001746      662e0f1f8400.  nop word cs:[rax + rax]
|           ; CODE XREF from sym.multiply1_simd_float__float__float (0x10000179f)
|       .-> 0x100001750      c5fc100407     vmovups ymm0, ymmword [rdi + rax]
|       :   0x100001755      c5fc590406     vmulps ymm0, ymm0, ymmword [rsi + rax]
|       :   0x10000175a      c5fc110402     vmovups ymmword [rdx + rax], ymm0
|       :   0x10000175f      c5fc10440720   vmovups ymm0, ymmword [rdi + rax + 0x20]
|       :   0x100001765      c5fc59440620   vmulps ymm0, ymm0, ymmword [rsi + rax + 0x20]
|       :   0x10000176b      c5fc11440220   vmovups ymmword [rdx + rax + 0x20], ymm0
|   

## 3 multiplication

The second set uses 3 multiplications:

```cpp
void multiply3_loop(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat*width; i+=width)
    {
        for (size_t j=i; j<i+width; ++j)
        {
            r[j] = a[j] * a[j];
            r[j] *= b[j];
            r[j] *= b[j];
        }
    }
}

void multiply3_simd(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat; ++i)
    {
        __m256 * ma = (__m256 *) (&a[i*width]);
        __m256 * mb = (__m256 *) (&b[i*width]);
        __m256 * mr = (__m256 *) (&r[i*width]);
        *mr = _mm256_mul_ps(*ma, *ma);
        *mr = _mm256_mul_ps(*mr, *mb);
        *mr = _mm256_mul_ps(*mr, *mb);
    }
}
```

In [7]:
# 3 multiplication with loop
!r2 -Aqc "e scr.color=0 ; s sym.multiply3_loop_float__float__float ; pdf" 01_mul/mul

            ;-- func.1000017b0:
/ (fcn) sym.multiply3_loop_float__float__float 352
|   sym.multiply3_loop_float__float__float ();
|           ; DATA XREF from entry0 (0x100002e48)
|           0x1000017b0      55             push rbp
|           0x1000017b1      4889e5         mov rbp, rsp
|           0x1000017b4      31c0           xor eax, eax
|           0x1000017b6      662e0f1f8400.  nop word cs:[rax + rax]
|           ; CODE XREF from sym.multiply3_loop_float__float__float (0x100001908)
|       .-> 0x1000017c0      c5fa100487     vmovss xmm0, dword [rdi + rax*4]
|       :   0x1000017c5      c5fa59c0       vmulss xmm0, xmm0, xmm0
|       :   0x1000017c9      c5fa110482     vmovss dword [rdx + rax*4], xmm0
|       :   0x1000017ce      c5fa590486     vmulss xmm0, xmm0, dword [rsi + rax*4]
|       :   0x1000017d3      c5fa110482     vmovss dword [rdx + rax*4], xmm0
|       :   0x1000017d8      c5fa590486     vmulss xmm0, xmm0, dword [rsi + rax*4]
|       :   0x1000017dd      c5fa11048

In [8]:
# 3 multiplication with simd/avx
!r2 -Aqc "e scr.color=0 ; s sym.multiply3_simd_float__float__float ; pdf" 01_mul/mul

            ;-- func.100001910:
/ (fcn) sym.multiply3_simd_float__float__float 107
|   sym.multiply3_simd_float__float__float ();
|           ; DATA XREF from entry0 (0x100002f47)
|           0x100001910      55             push rbp
|           0x100001911      4889e5         mov rbp, rsp
|           0x100001914      31c0           xor eax, eax
|           0x100001916      662e0f1f8400.  nop word cs:[rax + rax]
|           ; CODE XREF from sym.multiply3_simd_float__float__float (0x100001974)
|       .-> 0x100001920      c5fc100407     vmovups ymm0, ymmword [rdi + rax]
|       :   0x100001925      c5fc59c0       vmulps ymm0, ymm0, ymm0
|       :   0x100001929      c5fc110402     vmovups ymmword [rdx + rax], ymm0
|       :   0x10000192e      c5fc590406     vmulps ymm0, ymm0, ymmword [rsi + rax]
|       :   0x100001933      c5fc110402     vmovups ymmword [rdx + rax], ymm0
|       :   0x100001938      c5fc590406     vmulps ymm0, ymm0, ymmword [rsi + rax]
|       :   0x10000193d      c5fc11

## 5 multiplication

The third (last) set uses 5 multiplications:

```cpp
void multiply5_loop(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat*width; i+=width)
    {
        for (size_t j=i; j<i+width; ++j)
        {
            r[j] = a[j] * a[j];
            r[j] *= a[j];
            r[j] *= b[j];
            r[j] *= b[j];
            r[j] *= b[j];
        }
    }
}

void multiply5_simd(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat; ++i)
    {
        __m256 * ma = (__m256 *) (&a[i*width]);
        __m256 * mb = (__m256 *) (&b[i*width]);
        __m256 * mr = (__m256 *) (&r[i*width]);
        *mr = _mm256_mul_ps(*ma, *ma);
        *mr = _mm256_mul_ps(*mr, *ma);
        *mr = _mm256_mul_ps(*mr, *mb);
        *mr = _mm256_mul_ps(*mr, *mb);
        *mr = _mm256_mul_ps(*mr, *mb);
    }
}
```

In [9]:
# 5 multiplication with loop
!r2 -Aqc "e scr.color=0 ; s sym.multiply5_loop_float__float__float ; pdf" 01_mul/mul

            ;-- func.100001980:
/ (fcn) sym.multiply5_loop_float__float__float 540
|   sym.multiply5_loop_float__float__float ();
|           ; DATA XREF from entry0 (0x1000030b2)
|           0x100001980      55             push rbp
|           0x100001981      4889e5         mov rbp, rsp
|           0x100001984      31c0           xor eax, eax
|           0x100001986      662e0f1f8400.  nop word cs:[rax + rax]
|           ; CODE XREF from sym.multiply5_loop_float__float__float (0x100001b94)
|       .-> 0x100001990      c5fa100487     vmovss xmm0, dword [rdi + rax*4]
|       :   0x100001995      c5fa59c0       vmulss xmm0, xmm0, xmm0
|       :   0x100001999      c5fa110482     vmovss dword [rdx + rax*4], xmm0
|       :   0x10000199e      c5fa590487     vmulss xmm0, xmm0, dword [rdi + rax*4]
|       :   0x1000019a3      c5fa110482     vmovss dword [rdx + rax*4], xmm0
|       :   0x1000019a8      c5fa590486     vmulss xmm0, xmm0, dword [rsi + rax*4]
|       :   0x1000019ad      c5fa11048

In [10]:
# 5 multiplication with simd/avx
!r2 -Aqc "e scr.color=0 ; s sym.multiply5_simd_float__float__float ; pdf" 01_mul/mul

            ;-- func.100001ba0:
/ (fcn) sym.multiply5_simd_float__float__float 87
|   sym.multiply5_simd_float__float__float ();
|           ; DATA XREF from entry0 (0x1000031b1)
|           0x100001ba0      55             push rbp
|           0x100001ba1      4889e5         mov rbp, rsp
|           0x100001ba4      31c0           xor eax, eax
|           0x100001ba6      662e0f1f8400.  nop word cs:[rax + rax]
|           ; CODE XREF from sym.multiply5_simd_float__float__float (0x100001bf0)
|       .-> 0x100001bb0      c5fc100407     vmovups ymm0, ymmword [rdi + rax]
|       :   0x100001bb5      c5fc59c0       vmulps ymm0, ymm0, ymm0
|       :   0x100001bb9      c5fc110402     vmovups ymmword [rdx + rax], ymm0
|       :   0x100001bbe      c5fc590407     vmulps ymm0, ymm0, ymmword [rdi + rax]
|       :   0x100001bc3      c5fc110402     vmovups ymmword [rdx + rax], ymm0
|       :   0x100001bc8      c5fc590406     vmulps ymm0, ymm0, ymmword [rsi + rax]
|       :   0x100001bcd      c5fc110

# OpenMP

In [11]:
!make -C 03_omp clean ; make -C 03_omp ; 03_omp/omp

rm -rf *.o *.dSYM/ omp
clang++ -Xpreprocessor -fopenmp -std=c++17 -g -O3  -c -o omp.o omp.cpp
clang++ -Xpreprocessor -fopenmp -std=c++17 -g -O3   -lomp -o omp omp.o
Hello from thread 0, nthreads 4
Hello from thread 1, nthreads 4
Hello from thread 2, nthreads 4
Hello from thread 3, nthreads 4


# Exercises

1. Replace the single-precision floating-point vector type `__m256` with the double-precision floating-point vector type `__m256d` in the example, and compare the performance with the sinple-precision version.

# References

1. Crunching Numbers with AVX and AVX2 (AVX tutorials): https://www.codeproject.com/Articles/874396/Crunching-Numbers-with-AVX-and-AVX
2. Agner Fog (Agner's website): https://www.agner.org

   * Instruction table (latency information): https://www.agner.org/optimize/instruction_tables.pdf
   * Software optimization resources: https://www.agner.org/optimize/
3. x86 and amd64 instruction reference (unofficial) by Félix Cloutier: https://www.felixcloutier.com/x86/
4. Intel Intrinsics Guide: https://software.intel.com/sites/landingpage/IntrinsicsGuide/
5. Computer Organization and Assembly Languages by Yung-Yu Chuang, NTU: https://www.csie.ntu.edu.tw/~cyy/courses/assembly/12fall/news/